# **Classification du churn dans une entreprise de service numérique**

L'objectif est de prédire la résiliation des clients d'une entreprise de services téléphoniques et Internet à l'aide d'un modèle de Machine Learning.

- `CustomerID` : Un identifiant unique pour chaque client.
- `Zip Code` : Le code postal de la résidence principale du client.
- `Gender` : Le genre du client : Masculin, Féminin.
- `Senior Citizen` : Indique si le client a 65 ans ou plus : Oui, Non.
- `Partner` : Indique si le client a un partenaire : Oui, Non.
- `Dependents` : Indique si le client vit avec des personnes à charge : Oui, Non. Les personnes à charge peuvent être des enfants, parents, grands-parents, etc.
- `Tenure Months` : Indique le nombre total de mois que le client a passé avec l'entreprise à la fin du trimestre d'étude.
- `Phone Service` : Indique si le client est abonné à un service de téléphone à domicile avec l'entreprise : Oui, Non.
- `Multiple Lines` : Indique si le client est abonné à plusieurs lignes téléphoniques avec l'entreprise : Oui, Non.
- `Internet Service` : Indique si le client est abonné à un service Internet avec l'entreprise : Non, DSL, Fibre optique.
- `Online Security` : Indique si le client est abonné à un service supplémentaire de sécurité en ligne fourni par l'entreprise : Oui, Non.
- `Online Backup` : Indique si le client est abonné à un service supplémentaire de sauvegarde en ligne fourni par l'entreprise : Oui, Non.
- `Device Protection` : Indique si le client est abonné à un plan de protection supplémentaire pour son équipement Internet fourni par l'entreprise : Oui, Non.
- `Tech Support` : Indique si le client est abonné à un plan de support technique supplémentaire avec des temps d'attente réduits : Oui, Non.
- `Streaming TV` : Indique si le client utilise son service Internet pour diffuser des programmes télévisés via un fournisseur tiers : Oui, Non. L'entreprise ne facture pas de frais supplémentaires pour ce service.
- `Streaming Movies` : Indique si le client utilise son service Internet pour diffuser des films via un fournisseur tiers : Oui, Non. L'entreprise ne facture pas de frais supplémentaires pour ce service.
- `Contract` : Indique le type de contrat actuel du client : Mois par Mois, Un an, Deux ans.
- `Paperless Billing` : Indique si le client a opté pour la facturation sans papier : Oui, Non.
- `Payment Method` : Indique comment le client paye sa facture : Prélèvement bancaire, Carte de crédit, Chèque envoyé par courrier, Chèque automatique.
- `Monthly Charge` : Indique le montant total actuel mensuel des services de l'entreprise pour le client.
- `Total Charges` : Indique les frais totaux du client, calculés jusqu'à la fin du trimestre spécifié ci-dessus.
- `CLTV` : Valeur vie client (Customer Lifetime Value). Une CLTV prédite est calculée à l'aide de formules d'entreprise et de données existantes. Plus la valeur est élevée, plus le client est précieux. Les clients de grande valeur doivent être surveillés pour éviter leur départ.
- `Churn Value` : 1 = le client a quitté l'entreprise ce trimestre. 0 = le client est resté avec l'entreprise. Ceci est la variable à prédire.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
color_b, color_g, color_r = sns.color_palette()[:3]

## **0. Vision d'ensemble**

In [ ]:
df = pd.read_csv("DatasetChurn.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

- Les colonnes `Zip Code` et `Lat Long` ont le même nombre de valeurs uniques.
- Y'aurait-il une correspondance entre ces deux colonnes ?

In [ ]:
# Confirmons l'association entre un Zip Code et des coordonnées géographiques
(
    df.groupby("Zip Code")  # pour chaque Zip Code
    ["Lat Long"].agg(set)   # on récupère l'ensemble des coordonnées géographiques
    .apply(len)             # on compte le nombre d'éléments récupérés par ligne
    .unique()               # on récupère les valeurs uniques des précédents comptages
)

- Le résultat ci-dessus confirme l'association entre un Zip Code et des coordonnées géographiques, nous n'allons conserver qu'une seule de ces colonnes.
- Il serait plus intéressant de conserver la colonne `Lat Long` pour mieux détecter les tendances liés au positionnement géographique.
- Cette colonne sera ensuite splitée en `Lat` et `Long` pour une meilleure exploitation.

In [ ]:
df.drop(columns=["Zip Code"], inplace=True)
df["Lat"] = df["Lat Long"].str.split(", ").str[0].astype(float)
df["Long"] = df["Lat Long"].str.split(", ").str[1].astype(float)

- Observons les valeurs manquantes

In [ ]:
df.isna().sum()

In [ ]:
n_na = df.isna().any(axis=1).sum()
print(f"Nombre de lignes avec des valeurs manquantes : {n_na}")
print(f"Pourcentage de lignes avec des valeurs manquantes : {n_na / df.shape[0] * 100:.2f}%")

- Dans un premier temps, nous allons supprimer toutes les lignes concernées, car le nombre de valeurs manquantes est relativement faible.
- Par la suite, nous évaluerons s'il est possible de les imputer à l'aide de règles métier ou si une imputation est nécessaire, pourvu que l'impact sur le modèle soit significatif. L'objectif est de minimiser autant que possible toute altération des données.

In [ ]:
df.dropna(inplace=True)

- Maintenant nous catégorisons les variables selon qu'elles soient discrètes ou continues afin de faciliter l'analyse.

In [ ]:
id_col = ["CustomerID"]
disc = ["Gender", "Senior Citizen", "Partner", "Dependents", "Phone Service", "Multiple Lines", "Internet Service", "Online Security", "Online Backup", "Device Protection", "Tech Support", "Streaming TV", "Streaming Movies", "Contract", "Paperless Billing", "Payment Method"]
cont = ["Lat", "Long", "Monthly Charges", "Total Charges", "Tenure Months", "CLTV"]
target = "Churn Value"

## **1. Analyse des données**

In [ ]:
df[target].value_counts(normalize=True).round(2)

- On constate une répartition inégale de la variable cible : 16% des clients résilient leur contrat.

### **1.1. Variables discrètes**

In [ ]:
print(f"{len(disc)} variables discrètes : {disc}")

In [ ]:
def plot_disc_variables(df, cols, n_rows=2, n_cols=8, figsize=(24, 6), fig_title="Répartition des variables discètes", fig_title_font_size=16):
    _, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    
    if len(cols) > 1:
        axes = axes.flatten()
    
    for i, col in enumerate(cols):
        ax = axes[i] if len(cols) > 1 else axes

        sns.countplot(data=df, x=col, ax=ax)
        
        n = df[col].count()
        for p in ax.patches:
            # Pourcentages
            ax.text(
                p.get_x() + p.get_width() / 2,
                p.get_y() + p.get_height() / 2,
                f"{p.get_height() / n:.0%}",
                ha="center", va="center",
                c="w", weight="bold",
            )
            
            # Formatage des labels sur les axes
            ax.set_xlabel(col, weight="bold")
            ax.set_xticks(ax.get_xticks())
            
            xticklabels = ["\n".join(label.get_text().split()) for label in ax.get_xticklabels()]
            ax.set_xticklabels(xticklabels, rotation=0)
            
            ax.set_yticks([])
            ax.set_ylabel("")

    plt.suptitle(fig_title, size=fig_title_font_size, weight="bold")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_disc_variables(df=df, cols=disc)

- Les répartitions hommes/femmes et avec/sans partenaire sont très équilibrées, tandis que les répartitions des personnes agées et des personnes ayant souscrit à un service de téléphone mobile sont déséquilibrées en faveur respecivement des personnes de moins de 65 ans (85%), et des personnes ayant souscrit à un service de téléphone mobile (90%).
- On peut également noter que le quart des clients ne vit pas avec des personnes à charge et la moitié renouvelle mensuellement son abonnement, tandis que l'autre moitié est presque équitablement répartie entre les personnes ayant souscrit à un abonnement annuel et les personnes ayant souscrit à un abonnement sur deux ans.
- La variable `Multiple Lines` est une version détaillée de `Phone Service`, on mesurera l'impact de la suppression de cette dernière de notre jeu de données.
- Aussi, nous remarquons que près du quart des clients de l'échantillon n'est pas abonné à un service Internet. En séparant les données suivant cette variable, ce dernier groupe pourra se passer des 6 variables qui donnent davantage de précisions sur lesdits services : `Online Security`, `Online Backup`, `Device Protection`, `Tech Support`, `Streaming TV` et `Streaming Movies`.

#### **1.1.1. Séparation avec/sans Internet**

In [ ]:
df_internet = df[df["Internet Service"] != "No"].copy()
df_no_internet = df[df["Internet Service"] == "No"].copy()
disc_no_internet = ["Gender", "Senior Citizen", "Partner", "Dependents", "Phone Service", "Multiple Lines", "Contract", "Paperless Billing", "Payment Method"]

- Pour juger de la pertinence de ce split, nous évaluons le lien entre les autres variables (discrètes) et la variable cible à travers un test du chi-2.

In [ ]:
from scipy.stats import chi2_contingency

tmp_dfs = {
    "Avec Internet": df_internet,
    "Sans Internet": df_no_internet,
}

for col in disc_no_internet:
    print(f" {col} ".center(25, "-"))
    for df_name, tmp_df in tmp_dfs.items():
        pvalue = chi2_contingency(pd.crosstab(tmp_df[col], tmp_df[target])).pvalue
        lien = " *" if pvalue < 0.05 else ""
        print(f"- {df_name}".ljust(15) + f"  {pvalue:.4f}" + lien)
    print()

- On observe qu'effectivement, selon que les clients aient souscrit ou non à un service Internet, le soutien à l'hypothèse d'indépendance entre les variables varie de manière significative. Les test ci-dessus suggèrent, avec un seuil fixé à 2%, des liens de dépendance suivants avec la variable cible :

| Colonnes | Avec Internet | Sans Internet |
|-|:-:|:-:|
| Gender            |   |   |
| Senior Citizen    | x |   |
| Partner           | x | x |
| Dependents        | x | x |
| Phone Service     | x |RAS|
| Multiple Lines    | x | x |
| Contract          | x | x |
| Paperless Billing | x |   |
| Payment Method    | x | x |

- Le fait qu'il existe des différences aussi notoires entre ces deux groupes nous conforte dans ce choix de séparer ainsi les données.
- **RAS → Aucune interprétation** : Les clients n'ayant souscrit à aucun service Internet sont forcément des abonnés mobile, on n'a donc qu'une seule modalité dans cette classe (Yes).
- Pour davantage comprendre, analysons visuellement ces relations.

In [ ]:
def plot_disc_vs_target(df, segment, cols, figsize=(28, 4), fig_title_font_size=20):
    _, axes = plt.subplots(1, len(cols), figsize=figsize)
    
    for i, col in enumerate(cols):
        prop_df = pd.crosstab(df[col], df[target])
        prop_df = prop_df.div(prop_df.sum(axis=1), axis=0)
        
        ax = axes[i] if len(cols) > 1 else axes
        bar = prop_df.plot(kind="bar", stacked=True, legend=False, color=[color_g, color_r], alpha=0.8, ax=ax)
        
        # Pourcentages
        for i, p in enumerate(bar.patches):
            if i < len(bar.patches) / 2:
                bar.text(
                    p.get_x() + p.get_width() / 2,
                    p.get_y() + p.get_height() / 2,
                    f"{p.get_height():.0%}",
                    ha="center", va="center",
                    c="w", weight="bold",
                )
                
        # Formatage des labels sur les axes
        bar.set_xlabel(col, weight="bold")
        bar.set_xticks(bar.get_xticks())
        
        xticklabels = ["\n".join(label.get_text().split()) for label in ax.get_xticklabels()]
        bar.set_xticklabels(xticklabels, rotation=0)
        
        bar.set_yticks([])
        bar.set_ylabel("")

    ax = axes[0] if len(cols) > 1 else axes
    ax.legend(title=target, bbox_to_anchor=(0, 1))
    
    plt.suptitle(f"Taux de churn, Segment = {segment}", size=fig_title_font_size, weight="bold")
    plt.tight_layout()
    plt.show()

In [ ]:
for segment, tmp_df in tmp_dfs.items():
    plot_disc_vs_target(df=tmp_df, segment=segment, cols=disc_no_internet)

In [ ]:
print("Taux de churn :")
print(f"- Avec Internet : {df_internet[target].mean():.2%}")
print(f"- Sans Internet : {df_no_internet[target].mean():.2%}")

- Le premier constat est que la taux de churn est globalement plus élevé chez les clients ayant souscrit à des services Internet (20% vs. 4%).
- Précédemment, nous avons relevé des liens de dépendance entre certaines variables et la cible, nous pouvons dès lors apporter davantage de précisions.
- Pour l'ensemble des clients, nous remarquons que le taux de résiliation est plus élevé lorsque le client :
    - n'a pas de partenaire
    - n'a pas de personnes à charge
    - renouvelle mensuellement son contrat
    - ne possède qu'une seule ligne de téléphone
- Pour les cliens avec Internet, le taux de churn est plus élevé lorsque le client :
    - a plus de 65 ans
    - opte pour une facturation sans papier
    - règle ses factures par chèque électronique
- Pour ceux qui n'utilisent aucun service Internet, le churn est élevé lorsque le client :
    - règle ses factures via un chèque envoyé par courrier (impact relativement faible)
- Ceci nous aide à identifier ces variables comme des features importantes pour notre modèle.

#### **1.1.2. Analyse du lien entre les services Internet et la variable cible**

In [ ]:
internet_services = ["Online Security", "Online Backup", "Device Protection", "Tech Support", "Streaming TV", "Streaming Movies"]

# Nombre de services auxquels le client a souscrit
df_internet["Services Count"] = df_internet[internet_services].replace({"No": "0", "Yes": "1"}).astype(int).sum(axis=1)
internet_services = ["Services Count"] + internet_services

# Test du chi-2 pour juger de l'indépendance entre les variables
for col in internet_services:
    print(f" {col} ".center(25, "-"))
    pvalue = chi2_contingency(pd.crosstab(df_internet[col], df_internet[target])).pvalue
    lien = " *" if pvalue < 0.05 else ""
    print(f"- Internet".ljust(15) + f"  {pvalue:.4f}" + lien)
    print()

In [ ]:
# Graphiques pour une meilleure interprétation des résultats
plot_disc_vs_target(df=df_internet, segment="Internet", cols=internet_services)
plot_disc_variables(df=df_internet, cols=["Services Count"], n_rows=1, n_cols=1, figsize=(4, 3), fig_title="Répartition du nombre de services Internet", fig_title_font_size=12)

- Il semblerait qu'il existe une relation de dépendance entre les différents services Internet et le taux de churn : plus le client souscrit à des options, plus on a de chances qu'il reste.
- Cette tendance est moins marquée pour les services liés au Streaming (TV et films). Sachant d'après le descriptif des données que l'entreprise ne facture pas de frais supplémentaires pour ce service, nous pouvons comprendre pourquoi ces services ne constituent pas un facteur différenciant dans la décion de rester ou de partir.
- Serait-il plus pertinent de ne conserver que le nombre de services consommés (hormis les services de streaming) étant donné la tendance observée ?

In [ ]:
internet_services = ["Online Security", "Online Backup", "Device Protection", "Tech Support"]

# Nombre de services auxquels le client a souscrit
df_internet["Services Count"] = df_internet[internet_services].replace({"No": "0", "Yes": "1"}).astype(int).sum(axis=1)
internet_services = ["Services Count"] + internet_services

# Graphiques pour une meilleure interprétation des résultats
plot_disc_vs_target(df=df_internet, segment="Internet", cols=internet_services)
plot_disc_variables(df=df_internet, cols=["Services Count"], n_rows=1, n_cols=1, figsize=(4, 3), fig_title="Répartition du nombre de services Internet", fig_title_font_size=12)

- Les observations précédentes ne sont pas modifiées lorsque l'on supprime les colonnes liées au streaming, qu'il s'agisse de la relation décroissante entre le nombre de services et le taux de churn, ou alors de la répartition globale du nombre de services.

### **1.2. Variables continues**

In [ ]:
print(f"{len(cont)} variables continues : {cont}")

In [ ]:
cols = ["Monthly Charges", "Total Charges", "Tenure Months", "CLTV"]

In [ ]:

def plot_cont_vs_target(df, cols, target_value, segment, figsize=(18, 3)):
    _, axes = plt.subplots(1, len(cols), figsize=figsize)

    color = color_r if target_value else color_g
    for i, col in enumerate(cols):
        axes[i].hist(df.loc[df[target] == target_value, col], bins=100, color=color, alpha=0.8, label=target_value)
        axes[i].set_title(col)

    plt.suptitle(f"Distribution des variables continues\nSegment = {segment}, Churn Value = {target_value}", y=0.95, size=12, weight="bold")
    plt.tight_layout()
    plt.show()

def plot_cont_vs_cont(df, vars, segment):
    g = sns.PairGrid(df, hue=target, vars=vars, palette=[color_g, color_r], corner=True, diag_sharey=False)
    g.map_diag(sns.histplot, bins=100)
    g.map_lower(sns.scatterplot, s=10, alpha=0.8)
    g.add_legend(bbox_to_anchor=(0.92, 0.95))
    plt.suptitle(f"Relations entre les variables continues, Segment = {segment}", y=1.01, size=14, weight="bold")
    plt.show()

#### **1.2.1. Avec Internet**

In [ ]:
segment = "Avec Internet"
plot_cont_vs_target(df=df_internet, cols=cols, target_value=0, segment=segment)
plot_cont_vs_target(df=df_internet, cols=cols, target_value=1, segment=segment)

- Les montants mensuels et totaux dépensés par les clients semblent ne pas avoir d'impact réel sur le churn, même si on observe qu'un nombre important de clients qui partent ont de faibles charges totales.
- Au bout de 70 mois d'ancienneté, il devient très difficile de perdre des clients.
- Les clients qui résilient le font majoritairement dans les premiers mois.
- Cela suggère que les premiers mois sont critiques : si un client reste abonné au-delà d'une certaine période, il a moins de chances de partir.
- Les clients ayant une valeur vie > 6 000 ont une faible probabilité de résiliation.

In [ ]:
plot_cont_vs_cont(df=df_internet, vars=cols, segment=segment)

- Il semblerait que les données soient divisées en 2 groupes, cela s'observe notamment avec le lien entre la valeur client et (les charges totales ou l'ancienneté), mais plus précisément avec la dernière.

In [ ]:
plt.figure(figsize=(24, 5))
c1, c2 = ["Tenure Months", "CLTV"]

plt.subplot(1, 2, 1)
y1 = df_internet[[c1, c2]].groupby(c1).max(c2)
sns.lineplot(y1, marker=".")
plt.ylabel("max(CLTV)")
plt.title("Valeur client maximale pour chaque valeur d'ancienneté", weight="bold")

plt.subplot(1, 2, 2)
y2 = df_internet.loc[(df_internet[c2] > 3950) & (df_internet[c2] < 4050), [c1, c2]].groupby(c2).max(c1)
sns.lineplot(y2, marker=".")
plt.ylabel("max(Tenure Months)")
plt.title("Ancienneté maximale pour chaque valeur d'ancienneté", weight="bold")

plt.show()

- Le bond en termes :
    - de valeur client se situe entre le 45e et le 50e mois d'ancienneté,
    - d'ancienneté se situe autour d'une valeur client à 4000.

In [ ]:
display(y1.iloc[45:50])

thresh1 = y1.loc[48:49].mean()["CLTV"]
print(f"Seuil CLTV :", thresh1)

In [ ]:
display(y2.iloc[25:30])

thresh2 = y2.iloc[27:29].mean()["Tenure Months"]
print(f"Seuil Ancienneté :", thresh2)

##### **1.2.1.a. Avec Internet, G1**

In [ ]:
segment = "Avec Internet, G1"
df_internet_g1 = df_internet.query("(CLTV < @thresh1) & (`Tenure Months` <= @thresh2)").copy()
plot_cont_vs_cont(df=df_internet_g1, vars=cols, segment=segment)

- Ces graphiques nous donnent des raisons de penser qu'une segmentation efficace des clients en fonction du montant de leur facture mensuelle pourrait revéler des tendances intéressantes.

In [ ]:
# Les différences importantes entre les valeurs uniques et ordonnées des charges mensuelles constitueraient différents groupes d'intérêt
y_data = df_internet_g1["Monthly Charges"].drop_duplicates().sort_values().reset_index(drop=True)
y = y_data.diff()
x = np.arange(len(y))

ax = sns.lineplot(x=x, y=y, marker=".", lw=0.5)
plt.setp(ax.lines, markersize=5)
plt.title(f"Différences entre les valeurs uniques et ordonnées des charges mensuelles", weight="bold")
plt.ylabel(f"Delta Monthly Charges")
plt.show()

- Un coup d'oeil rapide nous permet de détecter une vingtaine de groupes, nous allons utiliser un algorithme de clustering pour les distinguer.

In [ ]:
from sklearn.cluster import KMeans

kmeans_internet = KMeans(n_clusters=20, random_state=42)
df_internet_g1["Monthly Charges Cluster"] = kmeans_internet.fit_predict(df_internet_g1[["Monthly Charges"]])

In [ ]:
# Les clusters sont rangés selon l'ordre croissant de leurs médianes pour garantir une progression logique
cluster_medians = df_internet_g1.groupby("Monthly Charges Cluster")["Monthly Charges"].median()
sorted_clusters = cluster_medians.sort_values().index 
clusters_mapping_no_internet = {old: new for new, old in enumerate(sorted_clusters)}
df_internet_g1["Monthly Charges Cluster"] = df_internet_g1["Monthly Charges Cluster"].map(clusters_mapping_no_internet)

# sns.boxplot(df_internet_g1, x="Monthly Charges Cluster", y="Monthly Charges")
# plt.title("Répartition des charges mensuelles par cluster", weight="bold")
# plt.show()

- La visualisation des relations entre les variables continues pour chacun des clusters nous a permis d'observer certains phénomènes intéressants.

- 1. Il existe au sein de chaque groupe une très forte correlation entre les variables `Tenure Months` et `Total Charges` et pourtant d'un point de vue global, le lien n'est pas aussi prononcé.

In [ ]:
from scipy.stats import pearsonr
c1 = "Total Charges"
c2 = "Tenure Months"

corr_all = pearsonr(df_internet_g1[c1], df_internet_g1[c2]).statistic
print(f"Corrélation totale : {corr_all:.4f}")

corr_clusters = []
for i in range(20):
    tmp_df = df_internet_g1.query("`Monthly Charges Cluster` == @i")
    corr_clusters.append(pearsonr(tmp_df[c1], tmp_df[c2]).statistic)
    
print(f"Moyenne des corrélations intra-groupes : {np.mean(corr_clusters):.4f}")

# Quelques exemples visuels
_, axes = plt.subplots(1, 6, sharey=True, figsize=(18, 3))

sns.scatterplot(data=df_internet_g1, x=c1, y=c2, s=10, ax=axes[0])
axes[0].text(axes[0].get_xlim()[1], 2, f"Corr = {corr_all:.4f}", ha="right", weight="bold")
axes[0].set_title(f"Cluster = all")

for _, i in enumerate([0, 5, 10, 15, 19]):
    ax = axes[_ + 1]
    
    tmp_df = df_internet_g1.query("`Monthly Charges Cluster` == @i")

    sns.scatterplot(data=tmp_df, x=c1, y=c2, s=10, ax=ax)
    ax.text(ax.get_xlim()[1], 2, f"Corr = {corr_clusters[i]:.4f}", ha="right", weight="bold")
    ax.set_title(f"Cluster = {i}")
    
plt.suptitle(f"Relations entre les variables '{c1}' et '{c2}' pour différents clusters\nSegment = {segment}, G1", y=0.95, weight="bold")
plt.tight_layout()
plt.show()

- 2. Dépendant les clusters, il est plus ou moins possible d'identifier un ensemble de valeurs de charges totales pour lesquelles on a de moins en moins de résiliations.

In [ ]:
c1 = "CLTV"
c2 = "Total Charges"

# Quelques exemples visuels
_, axes = plt.subplots(1, 6, sharey=True, figsize=(18, 4))

sns.scatterplot(data=df_internet_g1, x=c1, y=c2, s=10, hue=target, palette=[color_g, color_r], ax=axes[0])
axes[0].set_title(f"Cluster = all")
axes[0].legend(title=target, bbox_to_anchor=(-0.1, -0.1))

for _, i in enumerate([0, 5, 10, 15, 19]):
    ax = axes[_ + 1]

    tmp_df = df_internet_g1.query("`Monthly Charges Cluster` == @i")

    sns.scatterplot(data=tmp_df, x=c1, y=c2, s=10, hue=target, palette=[color_g, color_r], ax=ax)
    ax.set_title(f"Cluster = {i}")
    ax.legend("")
    
plt.suptitle(f"Relations entre les variables '{c1}' et '{c2}' pour différents clusters\nSegment = {segment}", y=0.95, weight="bold")
plt.tight_layout()
plt.show()

- Pour la modélisation, nous allons discrétiser la variable `Monthly Charges` selon les clusters calculés précédemment, et supprimer `Total Charges` pour éviter la redondance dans les données. Nous conserverons donc `Tenure Months` que nous pourrons par ailleurs séparer suivant les 2 groupes observés (ancienneté <= et > 48 mois) pour l'expérimentation.
- Vérifions la pertinence de ces actions sur le 2e sous-segment des clients ayant recours aux services Internet.

##### **1.2.1.b. Avec Internet, G2**

In [ ]:
segment = "Avec Internet, G2"
df_internet_g2 = df_internet.drop(df_internet_g1.index).copy()
# plot_cont_vs_cont(df=df_internet_g2, vars=cols, segment=segment)

In [ ]:
df_internet_g2["Monthly Charges Cluster"] = kmeans_internet.predict(df_internet_g2[["Monthly Charges"]])
df_internet_g2["Monthly Charges Cluster"] = df_internet_g2["Monthly Charges Cluster"].map(clusters_mapping_no_internet)

In [ ]:
c1 = "Total Charges"
c2 = "Tenure Months"

corr_all = pearsonr(df_internet_g2[c1], df_internet_g2[c2]).statistic
print(f"Corrélation totale : {corr_all:.4f}")

corr_clusters = []
for i in range(20):
    tmp_df = df_internet_g2.query("`Monthly Charges Cluster` == @i")
    
    if len(tmp_df) > 2:
        corr_clusters.append(pearsonr(tmp_df[c1], tmp_df[c2]).statistic)
    
print(f"Moyenne des corrélations intra-groupes : {np.mean(corr_clusters):.4f}")

- En appliquant le même processus de discrétisation de la variable `Monthly Charges`, on observe la même augmentation de la corrélation intra-groupes entre l'ancienneté et les charges totales.

#### **1.2.2. Sans Internet**

In [ ]:
segment = "Sans Internet"
plot_cont_vs_target(df=df_no_internet, cols=cols, target_value=0, segment=segment)
plot_cont_vs_target(df=df_no_internet, cols=cols, target_value=1, segment=segment)

- En guise de premier constat, on notera une séparation nette en 2 classes suivant le montant des charges mensuelles.
- Par ailleurs, les clients qui paient plus de 22$ sont moins susceptibles de partir.

In [ ]:
plot_cont_vs_cont(df=df_no_internet, vars=cols, segment=segment)

- Les différents groupes peuvent nettement être séparés suivant le montant de la facture mensuelle.
- On peut également remarquer à nouveau la corrélation évidente entre l'ancienneté et le montant des charges totales (potentiellement renforcée par une segmentation des charges mensuelles comme vu précédemment).
- Enfin, les 2 groupes d'ancienneté distingués précédemment (< et >= 49 mois) sont aussi visibles ici.

In [ ]:
kmeans_no_internet = KMeans(n_clusters=2, random_state=42)
df_no_internet["Monthly Charges Cluster"] = kmeans_no_internet.fit_predict(df_no_internet[["Monthly Charges"]])

In [ ]:
# Les clusters sont rangés selon l'ordre croissant de leurs médianes pour garantir une progression logique
cluster_medians = df_no_internet.groupby("Monthly Charges Cluster")["Monthly Charges"].median()
sorted_clusters = cluster_medians.sort_values().index
clusters_mapping_internet = {old: new for new, old in enumerate(sorted_clusters)}
df_no_internet["Monthly Charges Cluster"] = df_no_internet["Monthly Charges Cluster"].map(clusters_mapping_internet)

#### **1.2.3. Coordonnées géographiques**

In [ ]:
sns.scatterplot(df, x="Long", y="Lat", hue=target, s=10, alpha=0.5, palette=[color_g, color_r])
plt.show()

- Aucune observation particulière ne se dégage de cette figure.

#### **1.2.4. Autres analyses**

In [ ]:
def plot_cont_vs_disc(df, cols, disc, segment, figsize=(18, 3)):
    for val in df[disc].sort_values().unique():
        _, axes = plt.subplots(1, len(cols), figsize=figsize)

        for i, col in enumerate(cols):
            sns.histplot(df.loc[df[disc] == val, [col, target]], x=col, hue=target, hue_order=(1, 0), bins=100, stat="density", palette=[color_r, color_g], ax=axes[i])
            
            axes[i].set_ylabel("")
            axes[i].set_title(col)
            axes[i].legend("")

        plt.suptitle(f"Distribution des variables continues suivant la variable {col}\nSegment = {segment}, {disc} = {val}", y=0.95, size=12, weight="bold")
        plt.tight_layout()
        plt.show()

- Nous remarquerons ci-dessous que les moyens de paiement automatiques (carte de crédit et virement bancaire) présentent les mêmes caractéristiques chez les clients avec contrats Internet.

In [ ]:
# plot_cont_vs_disc(df=df_internet, cols=cols, disc="Payment Method", segment="Avec Internet", figsize=(18, 3))

In [ ]:
# Test d'égalité des moyennes dans les 2 groupes
from scipy.stats import ttest_ind

a = df_internet.loc[df_internet["Payment Method"] == "Credit card (automatic)", target]
b = df_internet.loc[df_internet["Payment Method"] == "Bank transfer (automatic)", target]

ttest_ind(a, b).pvalue

## **2. Préparation des données**

In [1]:
import pandas as pd

# Chargement
df = pd.read_csv("DatasetChurn.csv")

# Suppression des valeurs manquantes
df.dropna(inplace=True)

# Suppression de `Phone Service`
df.drop(columns="Phone Service", inplace=True)

# Séparation avec/sans Internet
df_internet = df.query("`Internet Service` != 'No'").copy()
df_no_internet = df.drop(df_internet.index).copy()



# --- Avec Internet

# Suppression de Gender
df_internet.drop(columns=["Gender"], inplace=True)

# Supression des services de streaming
df_internet.drop(columns=["Streaming TV", "Streaming Movies"], inplace=True)

# Ajout de la colonne `Services Count` pour compter le nombre de services auxquels le client a souscrit
internet_services = ["Online Security", "Online Backup", "Device Protection", "Tech Support"]
df_internet["Services Count"] = df_internet[internet_services].replace({"No": "0", "Yes": "1"}).astype(int).sum(axis=1)

# Suppression des colonnes relatives aux différents services Internet
df_internet.drop(columns=internet_services, inplace=True)

# Clusters de `Monthly Charges`
from sklearn.cluster import KMeans

kmeans_internet = KMeans(n_clusters=20, random_state=42)
df_internet["Monthly Charges Cluster"] = kmeans_internet.fit_predict(df_internet[["Monthly Charges"]])

# Les clusters sont rangés selon l'ordre croissant de leurs médianes pour garantir une progression logique
cluster_medians = df_internet.groupby("Monthly Charges Cluster")["Monthly Charges"].median()
sorted_clusters = cluster_medians.sort_values().index 
clusters_mapping_internet = {old: new for new, old in enumerate(sorted_clusters)}
df_internet["Monthly Charges Cluster"] = df_internet["Monthly Charges Cluster"].map(clusters_mapping_internet)

# Suppression de `Monthly Charges` ???
# df_internet.drop(columns="Monthly Charges", inplace=True)

# Suppression de `Total Charges`
df_internet.drop(columns="Total Charges", inplace=True)

# Dicrétisation de `Tenure Months`
df_internet["Tenure Months Discrete"] = (df_internet["Tenure Months"] <= 48).astype(int)

# Suppression de `Tenure Months` ???
# df_internet.drop(columns="Tenure Months", inplace=True)

# Groupement des moyens de paiement automatiques
mask = (df_internet["Payment Method"].isin(["Bank transfer (automatic)", "Credit card (automatic)"]))
df_internet.loc[mask, "Payment Method"] = "Automatic"



# --- Sans Internet

# Suppression des services Internet
df_no_internet.drop(columns=["Internet Service", "Online Security", "Online Backup", "Device Protection", "Tech Support", "Streaming TV", "Streaming Movies"], inplace=True)

# Suppression de Gender
df_no_internet.drop(columns=["Gender"], inplace=True)

# Suppression de `Senior Citzen` et `Paperless Billing`
df_no_internet.drop(columns=["Senior Citizen", "Paperless Billing"], inplace=True)

# Clusters de `Monthly Charges`
from sklearn.cluster import KMeans

kmeans_no_internet = KMeans(n_clusters=2, random_state=42)
df_no_internet["Monthly Charges Cluster"] = kmeans_no_internet.fit_predict(df_no_internet[["Monthly Charges"]])

# Les clusters sont rangés selon l'ordre croissant de leurs médianes pour garantir une progression logique
cluster_medians = df_no_internet.groupby("Monthly Charges Cluster")["Monthly Charges"].median()
sorted_clusters = cluster_medians.sort_values().index 
clusters_mapping_no_internet = {old: new for new, old in enumerate(sorted_clusters)}
df_no_internet["Monthly Charges Cluster"] = df_no_internet["Monthly Charges Cluster"].map(clusters_mapping_no_internet)

# Suppression de `Monthly Charges` ???
# df_no_internet.drop(columns="Monthly Charges", inplace=True)

# Suppression de `Total Charges`
df_no_internet.drop(columns="Total Charges", inplace=True)

# Dicrétisation de `Tenure Months`
df_no_internet["Tenure Months Discrete"] = (df_no_internet["Tenure Months"] <= 48).astype(int)

# Suppression de `Tenure Months` ???
# df_no_internet.drop(columns="Tenure Months", inplace=True)

c:\Users\tchak\anaconda3\envs\dev\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


## **3. Model**

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import recall_score, classification_report
from xgboost import XGBClassifier

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

### **3.1. Avec Internet**

In [3]:
disc = ["Senior Citizen", "Partner", "Dependents", "Multiple Lines", "Internet Service", "Contract", "Paperless Billing", "Payment Method"]
cont = ["Tenure Months", "Monthly Charges", "CLTV", "Services Count", "Monthly Charges Cluster",]
misc = ["Tenure Months Discrete"]
targ = "Churn Value"

X = df_internet.drop(columns=targ)
y = df_internet[targ]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

preprocessor = ColumnTransformer(
    transformers=[
        ("disc", OneHotEncoder(drop="first", handle_unknown="ignore"), disc),
        ("cont", MinMaxScaler(), cont),
        ("pass", "passthrough", misc),
    ],
    remainder="drop"
)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

clf = XGBClassifier(random_state=42)
cross_val_score(clf, X_train_preprocessed, y_train, scoring="recall", cv=5).mean()

AttributeError: 'super' object has no attribute '__sklearn_tags__'

### **3.2. Sans Internet**

In [4]:
disc = ["Partner", "Dependents", "Multiple Lines", "Contract", "Payment Method"]
cont = ["Tenure Months", "Monthly Charges", "CLTV"]
misc = ["Monthly Charges Cluster", "Tenure Months Discrete"]

X = df_no_internet.drop(columns=targ)
y = df_no_internet[targ]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

preprocessor = ColumnTransformer(
    transformers=[
        ("disc", OneHotEncoder(drop="first", handle_unknown="ignore"), disc),
        ("cont", MinMaxScaler(), cont),
        ("pass", "passthrough", misc),
    ],
    remainder="drop"
)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

clf = XGBClassifier(random_state=42)
cross_val_score(clf, X_train_preprocessed, y_train, scoring="recall", cv=5).mean()

AttributeError: 'super' object has no attribute '__sklearn_tags__'